In [1]:
import pandas as pd

# Read data from IMTC

In [2]:
data1 = pd.read_csv('/home/imtc/Documenti/tmp/ID_DB_BRUGADA.csv', sep='\t')
# Drop incremental integer
data1.drop(['Unnamed: 23','Unnamed: 24'], axis=1, inplace=True)
# Drop row if missing 'Cognome' or 'Nome'
data1.dropna(subset=['Cognome', 'Nome'], inplace=True)
# Standardize column names
data1.columns = data1.columns.str.lower().str.replace(" ", "_")
# Drop completely empty columns (not almost empty ones)
data1 = data1.dropna(axis=1, how="all")
# Standardize columns: Convert to title case
data1["cognome"] = data1["cognome"].str.title().str.strip()
data1["nome"] = data1["nome"].str.title().str.strip()
# Recreate 'sex' column based on 'm' and 'f' values
data1["sex"] = data1.apply(lambda row: "M" if row["m"] == "X" else ("F" if row["f"] == "X" else None), axis=1)
data1.drop(columns=["m", "f"], inplace=True)  # Remove old columns
# Standardize date format (Data di Nascita -> date_of_birth)
data1["data_di_nascita"] = pd.to_datetime(data1["data_di_nascita"], errors="coerce", dayfirst=True)

In [3]:
data1.head()

,pos1_neg0,_procedure,cognome,nome,data_di_nascita,primo_ingresso,vbrdg,brdg,brdg_bis,bsee+,...,vpa,pa1,pa2,pa3,pa4,pa5,pa6,pa_bis,tot_procedure,sex
0,1,BrDg2467 Br1136 BrPA2283-2666,Abad,Rico,1990-10-30,11/01/2023,NaN,2467,NaN,NaN,...,NaN,2283,2666,NaN,NaN,NaN,NaN,NaN,4.0,M
1,1,BrDg1590 Br822 BrPA1455-1796-2174,Abate,Adriano,1979-04-08,29/03/2021,NaN,1590,NaN,NaN,...,NaN,1455,1796,2174.0,NaN,NaN,NaN,NaN,5.0,M
2,1,Br126 (BrPA76-457),Abbonizio,Filomena Silvana,1959-10-18,22/03/2017,NaN,NaN,NaN,NaN,...,NaN,76,457,NaN,NaN,NaN,NaN,NaN,3.0,F
3,1,BrDg2598,Abbruzzese,Maria Cristina,1995-09-13,17/04/2023,NaN,2598,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,F
4,1,BrDg2599 Br1213 BrPA2500-2849,Abbruzzese,Alfredo,1992-07-27,17/04/2023,NaN,2599,NaN,NaN,...,NaN,2500,2849,NaN,NaN,NaN,NaN,NaN,4.0,M


# Read data from EF

In [4]:
data2 = pd.read_csv('/home/imtc/Documenti/tmp/DATABASE_Clinica_pazienti_Brugada.csv', sep='\t')
data2.drop(['Unnamed: 232', 'Unnamed: 233'], axis=1, inplace=True)
# Standardize column names: Convert to lowercase and replace spaces with underscores
data2.columns = data2.columns.str.lower().str.replace(" ", "_")
# Drop completely empty columns
data2 = data2.dropna(axis=1, how="all")
data2["first_name"] = data2["first_name"].str.title().str.strip()
data2["last_name"] = data2["last_name"].str.title().str.strip()
data2["nazione_di_nascita"] = data2["nazione_di_nascita"].str.title().str.strip()
data2["regione_di_nascita"] = data2["regione_di_nascita"].str.title().str.strip()
data2["provincia_di_nascita"] = data2["provincia_di_nascita"].str.title().str.strip()
# Standardize date format (DATE OF BIRTH -> data_di_nascita, TEST DATE -> test_date)
data2["date_of_birth"] = pd.to_datetime(data2["date_of_birth"], errors="coerce", format="%m/%d/%Y", dayfirst=True)
data2["_test_date"] = pd.to_datetime(data2["_test_date"], errors="coerce", format="%m/%d/%Y", dayfirst=True)
# Ensure numeric columns are properly converted (e.g., age)
data2["age"] = pd.to_numeric(data2["age"], errors="coerce")

/tmp/ipykernel_2554844/478963137.py:1: DtypeWarning: Columns (0,2,13,17,18,21,22,23,24,25,26,27,28,32,34,36,42,44,50,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,75,76,77,78,79,80,86,87,88,91,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,148,150,162,164,165,169,196,212,219,220,227,228,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv('/home/imtc/Documenti/tmp/DATABASE_Clinica_pazienti_Brugada.csv', sep='\t')


In [5]:
data2.head()

,id,genlab_(y/n),pk_paziente_______id_cardioref,last_name,first_name,sex,date_of_birth,age,_test_date,nazione_di_nascita,...,described_previously_in_literature.2,gene_4,mutation_4,amino_acid_change_4,zygosity_4_(oz_e_hz),"significance_(1=benign,_2=likely_benign,_3=uncertain,_4=likely_pathogenic,_5=pathogenic).2",described_previously_in_literature.3,"inheritance_4_(maternal,_paternal,_de_novo)",note,id_randomized
0,3347,N,395720201,Abad,Rico,M,1990-10-30,32.2,2023-01-11,Filippine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2411,Y,302725861,Abate,Adriano,M,1979-04-08,42.0,2021-03-29,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,455,N,227356161,Abbonizio,Filomena Silvana,F,1959-10-18,57.4,2017-03-21,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevato rischio aritmico,NaN
3,3490,N,387005901,Abbruzzese,Maria Cristina,F,1965-09-13,57.6,2023-04-17,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3491,N,404687001,Abbruzzese,Alfredo,M,1992-07-27,30.7,2023-04-17,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Check Anagrafica

In [6]:
clinic_data1 = data1[['pos1_neg0', '_procedure', 'cognome', 'nome', 'data_di_nascita',
       'primo_ingresso', 'sex']]
clinic_data1.columns = ['ajmaline', 'id', 'last_name', 'first_name', 'birth_date',
       'registered_on', 'sex']

In [7]:
clinic_data2 = data2[['id','pk_paziente_______id_cardioref','last_name','first_name','sex','date_of_birth',
    '_test_date','nazione_di_nascita','regione_di_nascita',
    'provincia_di_nascita','proband_/_relatives',
    'fin_genetica_progressivo','ajmaline_test_result',
    'brs_baseline_pattern']]
clinic_data2.columns = ['id','id_cardioref','last_name','first_name','sex','birth_date',
    'test_date','nation','region',
    'province','family_status',
    'fin','ajmaline',
    'brs_baseline_pattern']

In [8]:
clinic_merged = clinic_data1.merge(clinic_data2, on=['first_name', 'last_name', 'birth_date', 'sex'], how='outer', indicator=True, suffixes=['_IMTC','_EF'])
for col in ["birth_date", "test_date", "registered_on"]:
    clinic_merged[col] = pd.to_datetime(clinic_merged[col], errors='coerce').dt.strftime('%Y-%m-%d')

/tmp/ipykernel_2554844/1949463787.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clinic_merged[col] = pd.to_datetime(clinic_merged[col], errors='coerce').dt.strftime('%Y-%m-%d')


In [9]:
clinic_merged.drop_duplicates(subset=['id_cardioref', 'first_name', 'last_name', 'birth_date', 'sex'], inplace=True)

In [10]:
clinic_merged

,ajmaline_IMTC,id_IMTC,last_name,first_name,birth_date,registered_on,sex,id_EF,id_cardioref,test_date,nation,region,province,family_status,fin,ajmaline_EF,brs_baseline_pattern,_merge
0,1,BrDg2467 Br1136 BrPA2283-2666,Abad,Rico,1990-10-30,2023-11-01,M,3347,395720201,2023-01-11,Filippine,NaN,NaN,P,1179,POSITIVE,SOSPETTO,both
1,1,BrDg1590 Br822 BrPA1455-1796-2174,Abate,Adriano,1979-04-08,2021-03-29,M,2411,302725861,2021-03-29,Italia,Piemonte,Torino,P,860,POSITIVE,1,both
2,1,Br126 (BrPA76-457),Abbonizio,Filomena Silvana,1959-10-18,2017-03-22,F,455,227356161,2017-03-21,Italia,Abruzzo,Chieti,P,20,POSITIVE,1,both
3,1,BrDg2598,Abbruzzese,Maria Cristina,1995-09-13,2023-04-17,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,1,BrDg2599 Br1213 BrPA2500-2849,Abbruzzese,Alfredo,1992-07-27,2023-04-17,M,3491,404687001,2023-04-17,Italia,Lombardia,Brescia,R,735,POSITIVE,SOSPETTO,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,NaN,NaN,Zanoli,Stefano,1975-12-01,NaN,M,4184,475594991.0,2024-10-02,Italia,Emilia Romagna,Modena,P,1435.0,POSITIVE,1,right_only
4919,NaN,NaN,Zec,Snezana,1977-03-18,NaN,F,1162,249113111.0,2018-10-01,Bosnia Erzegovina,NaN,NaN,P,244.0,POSITIVE,2,right_only
4920,NaN,NaN,Zeppetelli,Andrea,2002-05-20,NaN,M,1088,244502070.0,2018-07-26,NaN,NaN,NaN,R,69.0,NEGATIVE,SOSPETTO,right_only
4921,NaN,NaN,Zucca,Franco,1964-10-04,NaN,M,927,243398761.0,2018-04-22,NaN,NaN,NaN,NaN,NaN,NEGATIVE,SOSPETTO,right_only


In [11]:
clinic_merged[['_merge', 'id_IMTC','id_EF', 'id_cardioref', 'fin', 'last_name', 'first_name', 'birth_date', 'sex', 'nation', 'region',
       'province', 'family_status','registered_on', 'test_date', 'ajmaline_IMTC', 'ajmaline_EF',
       'brs_baseline_pattern']].to_csv('/home/imtc/Scaricati/clinical_imtc_ef_merged_TMP.csv', index=False)

In [12]:
# No birthdate

In [13]:
id_IMTC_left = list(clinic_merged[clinic_merged['_merge']!='both']['id_IMTC'].dropna())
id_EF_left = list(clinic_merged[clinic_merged['_merge']!='both']['id_EF'].dropna())

In [14]:
clinic_data1_left = clinic_data1[clinic_data1['id'].isin(id_IMTC_left)]
clinic_data2_left = clinic_data2[clinic_data2['id'].isin(id_EF_left)]

In [15]:
clinic_merged_left = clinic_data1_left.merge(clinic_data2_left, on=['first_name', 'last_name', 'sex'], how='outer', indicator=True, suffixes=['_IMTC','_EF'])

In [16]:
clinic_merged_left[['_merge', 'id_IMTC','id_EF', 'id_cardioref', 'fin', 'last_name', 'first_name', 'birth_date_EF','birth_date_IMTC', 'sex', 'nation', 'region',
       'province', 'family_status','registered_on', 'test_date', 'ajmaline_IMTC', 'ajmaline_EF',
       'brs_baseline_pattern']].to_csv('/home/imtc/Scaricati/clinical_imtc_ef_merged_TMP_2.csv', index=False)

In [17]:
# No first name

In [18]:
id_IMTC_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_IMTC'].dropna())
id_EF_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_EF'].dropna())

In [19]:
clinic_data1_left = clinic_data1_left[clinic_data1_left['id'].isin(id_IMTC_left)]
clinic_data2_left = clinic_data2_left[clinic_data2_left['id'].isin(id_EF_left)]

In [20]:
clinic_merged_left = clinic_data1_left.merge(clinic_data2_left, on=['last_name', 'birth_date', 'sex'], how='outer', indicator=True, suffixes=['_IMTC','_EF'])

In [21]:
clinic_merged_left[['_merge', 'id_IMTC','id_EF', 'id_cardioref', 'fin', 'last_name', 'first_name_EF', 'first_name_IMTC', 'birth_date', 'sex', 'nation', 'region',
       'province', 'family_status','registered_on', 'test_date', 'ajmaline_IMTC', 'ajmaline_EF',
       'brs_baseline_pattern']].to_csv('/home/imtc/Scaricati/clinical_imtc_ef_merged_TMP_3.csv', index=False)

In [22]:
# No last name

In [23]:
id_IMTC_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_IMTC'].dropna())
id_EF_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_EF'].dropna())

In [24]:
clinic_data1_left = clinic_data1_left[clinic_data1_left['id'].isin(id_IMTC_left)]
clinic_data2_left = clinic_data2_left[clinic_data2_left['id'].isin(id_EF_left)]

In [25]:
clinic_merged_left = clinic_data1_left.merge(clinic_data2_left, on=['first_name', 'birth_date', 'sex'], how='outer', indicator=True, suffixes=['_IMTC','_EF'])

In [26]:
clinic_merged_left[['_merge', 'id_IMTC','id_EF', 'id_cardioref', 'fin', 'first_name', 'last_name_EF', 'last_name_IMTC', 'birth_date', 'sex', 'nation', 'region',
       'province', 'family_status','registered_on', 'test_date', 'ajmaline_IMTC', 'ajmaline_EF',
       'brs_baseline_pattern']].to_csv('/home/imtc/Scaricati/clinical_imtc_ef_merged_TMP_4.csv', index=False)

In [27]:
# No sex

In [28]:
id_IMTC_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_IMTC'].dropna())
id_EF_left = list(clinic_merged_left[clinic_merged_left['_merge']!='both']['id_EF'].dropna())

In [29]:
clinic_data1_left = clinic_data1_left[clinic_data1_left['id'].isin(id_IMTC_left)]
clinic_data2_left = clinic_data2_left[clinic_data2_left['id'].isin(id_EF_left)]

In [30]:
clinic_merged_left = clinic_data1_left.merge(clinic_data2_left, on=['last_name', 'first_name', 'birth_date'], how='outer', indicator=True, suffixes=['_IMTC','_EF'])

In [31]:
clinic_merged_left[['_merge', 'id_IMTC','id_EF', 'id_cardioref', 'fin', 'first_name', 'last_name', 'birth_date', 'sex_EF', 'sex_IMTC', 'nation', 'region',
       'province', 'family_status','registered_on', 'test_date', 'ajmaline_IMTC', 'ajmaline_EF',
       'brs_baseline_pattern']].to_csv('/home/imtc/Scaricati/clinical_imtc_ef_merged_TMP_5.csv', index=False)

# SAMPLES

In [169]:
# Load Patient data
et_patients = pd.read_csv('/home/imtc/Documenti/tmp/easytrack/patient.csv')
et_patients.drop(['Id', 'Death Date', 'Death Cause', 'SSN', 'Birth Place'], axis=1, inplace=True)

In [170]:
# Load samples data
et_samples = pd.read_csv('/home/imtc/Documenti/tmp/easytrack/primary_samples_Jan2017_Aug2024.csv')
et_samples.drop(['Primary material name','Unnamed: 9', 'Id', 'Primary Material ID', 'Lab Id', 'Patient Id'], axis=1, inplace=True)

In [171]:
et_joined = et_patients.merge(et_samples, right_on='Patient Code', left_on='Code', suffixes=['_patient', '_psample'])
et_joined.to_csv('/home/imtc/Documenti/tmp/easytrack/joined_patient_psample.csv')

In [172]:
et_joined.columns = ['Code_patient', 'Date of Birth', 'Sex', 'First Name', 'Last Name',
       'Code_psample', 'Collection Date', 'Patient Code','Lab name',
       'Primary Material Code']

In [173]:
# Load omics_db
# Load samples data
odb_samples = pd.read_csv('/home/imtc/Documenti/tmp/omics_db_patients.csv')

In [174]:
merged_samples_patients = odb_samples.merge(et_joined, on=['First Name', 'Last Name', 'Date of Birth'], how='outer', suffixes=['_ODB', '_ET'], indicator=True)

In [175]:
merged_samples_patients.to_csv('/home/imtc/Documenti/tmp/easytrack/odb_et_patient_psample.csv', index=False)

# ANALYSIS

In [134]:
# Load Patient data
et_analysis = pd.read_csv('/home/imtc/Documenti/projects/omics_portal/db_tools/wes_rna_runs.csv')
# et_patients.drop(['Id', 'Death Date', 'Death Cause', 'SSN', 'Birth Place'], axis=1, inplace=True)

In [135]:
# Load samples data
et_samples = pd.read_csv('/home/imtc/Documenti/projects/omics_portal/db_tools/omics_db_patients_samples_data.csv')
# et_samples.drop(['Primary material name','Lab name','Unnamed: 9', 'Id', 'Primary Material ID', 'Lab Id', 'Patient Id'], axis=1, inplace=True)

In [136]:
et_analysis

,Library,Tech,Data Corsa,description,S1,S2,S3,S4,S5,S6,...,S39,S40,S41,S42,S43,S44,S45,S46,S47,S48
0,1,RNAseq,2/14/2022,Training - Serie 1 AC16 Marco,1H_IR,1H_IR_+_DOXY,3H_IR,3H_IR_+_DOXY,AC16 LVX N3 CT 12h Ischemia,AC16 LVX N3 CT + doxy 12h Ischemia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,RNAseq,3/15/2022,Serie 2 AC16 Marco,AC16 LVX N3 CT 12h Ischemia,AC16 LVX N3 CT + doxy 12h Ischemia,AC16 LVX N3 12h Ischemia,AC16 LVX N3 12h Ischemia + doxy,AC16 LVX N3 CT 1h IR,AC16 LVX N3 CT + doxy 1h IR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,RNAseq,4/13/2022,Serie 3 AC16 Marco,CT_12H_ISCHEMIA,CT_12H_ISCHEMIA_+_DOXY,12H_ISCHEMIA,12H_ISCHEMIA_+_DOXY,CT_1H_IR,CT_1H_IR_+_DOXY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,RNAseq,NaN,Serie 1 Marco + DG-PA,AC16 LVX N3 CT 1h IR,AC16 LVX N3 CT + doxy 1h IR,AC16 LVX N3 CT 3h IR,AC16 LVX N3 CT + doxy 3h IR,AC16 LVX N3 CT 6h IR,AC16 LVX N3 CT + doxy 6h IR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,WES,3/28/2022,Training - EM,EM_39,EM_40,EM_41,EM_42,EM_43,EM_44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,WES,4/15/2022,Br-Dg Auto Ab,BrDg2019,dg1969,br898,br910,dg1936,br930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,RNAseq,5/9/2022,NEW PBMCs,Dg1775,Dg1776,Dg1777,Dg1792,Dg1793,Dg1794,...,Dg2073,Dg2072,Dg2051,Dg2050,Dg2049,Dg2048,Dg2024,Dg2077,Dg2013,Dg2011
7,9,RNAseq,5/12/2022,NEW PBMCs,Dg1800,Dg1804,Dg1799,Dg1798,Dg1797,Dg1796,...,Dg567,Dg568,Dg563,Dg444,Dg446,Dg421,Dg417,Dg371,Dg400,Dg405
8,10,RNAseq,7/13/2022,Pre-Post abl,BrDg1539 (Pz814),BrPA1431 (Pz814),BrDg1586 (Pz817),BrPA1457 (Pz817),BrDg1587 (Pz818),BrPA1464 (Pz818),...,BrDg804 (Pz532),BrPA816 (Pz532),BrDg765 (Pz541),BrPA829 (Pz541),BrDg658 (Pz508),BrPA930 (Pz508),BrDg1590 (Pz822),BrDg1611 (Pz826),BrDg1409 (Pz739),BrPA1305 (Pz739)
9,11,WES,6/22/2022,Br-Dg Auto Ab + EM,Dg1603,Dg1679,Dg1695,Dg1844,Dg1928,Dg1940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
